<a href="https://colab.research.google.com/github/JonahLeggett/ML_Algo_Options_Trading/blob/main/AMCNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
!pip install hvplot
!pip install python-dotenv
!pip install alpaca_trade_api
!pip install yfinance
!pip install sentiment-investor
import json 
import requests
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import hvplot.pandas
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import tensorflow as tf
from sklearn.model_selection import train_test_split
import yfinance as yf

In [8]:
# Load .env environment variables 
load_dotenv()

# Set Alpaca API key and secret key
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
type(alpaca_secret_key )

# Create the Alpaca REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [9]:
# Get daily price data for AMC over the last 4 years.
barset = alpaca.get_barset('AMC', 'day', limit=1000).df
amc_df = barset['AMC']
display(amc_df.head())
display(amc_df.tail())

,open,high,low,close,volume
time,,,,,
2017-07-21 00:00:00-04:00,19.466,20.790,19.134,20.325,1100609
2017-07-24 00:00:00-04:00,20.400,20.717,19.158,20.500,1372608
2017-07-25 00:00:00-04:00,20.500,20.950,19.584,19.726,1267668
2017-07-26 00:00:00-04:00,19.679,21.175,19.538,19.915,1408162
2017-07-27 00:00:00-04:00,20.060,21.050,19.442,20.650,1815436


,open,high,low,close,volume
time,,,,,
2021-07-06 00:00:00-04:00,53.45,55.07,49.70,49.960,59214652
2021-07-07 00:00:00-04:00,47.66,48.99,42.80,45.070,90955448
2021-07-08 00:00:00-04:00,40.95,49.79,38.76,47.875,135822731
2021-07-09 00:00:00-04:00,48.35,48.92,45.82,46.180,55037860
2021-07-12 00:00:00-04:00,44.39,46.55,42.07,42.480,59429487


In [10]:
url = "http://sentimentinvestor.com/api/v3/quote?symbol=AMC&token=Sl8Sn4UQMiRGzFEQaIw6&key=698740758823221"

In [11]:
response = requests.get(url).json()

In [13]:
sa = json.dumps(response, indent=4, sort_keys=True)
print(data)

{
    "AHI": 2.283158866995074,
    "RHI": 0.9596907178695347,
    "SGP": 0.9702493211704698,
    "last_update": 1613693837.6303163,
    "reddit_comment_mentions": 37,
    "reddit_comment_sentiment": 0.52,
    "reddit_post_mentions": 0,
    "reddit_post_sentiment": null,
    "sentiment": 0.6588737816716181,
    "stocktwits_mentions": 108,
    "stocktwits_sentiment": 0.7627118644067796,
    "ticker": "AMC",
    "tweet_mentions": 284,
    "tweet_sentiment": 0.7128712871287128,
    "yahoo_finance_mentions": 501,
    "yahoo_finance_sentiment": 0.6328671328671329
}


In [17]:
sa_df = pd.DataFrame(sa)

ValueError: ignored

In [6]:
# Define the target set by selecting the 'close' column
y = amc_df["close"]

In [8]:
# Define features set X by selecting all columns except 'close'
X = amc_df.drop(columns=["close"])

In [9]:
# Create training and testing datasets using train_teat_split
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
# Create the StandardScaler instance
X_scaler = StandardScaler()

In [11]:
# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [12]:
# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the the number of inputs to the model
number_inputs = 4

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 3

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 2

# Create the Sequential model instance
nn = Sequential()

# Add the first Dense layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_inputs, activation="relu"))

# Add the second Dense layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(1, activation="linear"))

In [14]:
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 15        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the Sequential model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["acc"])

In [16]:
# Fit the model using 100 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
24/24 [==============================] - 1s 1ms/step - loss: 223.2086 - acc: 0.0000e+00
Epoch 2/100
24/24 [==============================] - 0s 2ms/step - loss: 221.5471 - acc: 0.0000e+00
Epoch 3/100
24/24 [==============================] - 0s 1ms/step - loss: 220.1446 - acc: 0.0000e+00
Epoch 4/100
24/24 [==============================] - 0s 2ms/step - loss: 219.0948 - acc: 0.0000e+00
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 218.2187 - acc: 0.0000e+00
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 217.4522 - acc: 0.0000e+00
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 216.7453 - acc: 0.0000e+00
Epoch 8/100
24/24 [==============================] - 0s 2ms/step - loss: 216.0664 - acc: 0.0000e+00
Epoch 9/100
24/24 [==============================] - 0s 2ms/step - loss: 215.4214 - acc: 0.0000e+00
Epoch 10/100
24/24 [==============================] - 0s 2ms/step - loss: 214.8004 - acc: 0.0000e+00

KeyboardInterrupt: ignored

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")